<a href="https://colab.research.google.com/github/VishnuTanpure/vishnurepository/blob/master/Microsoft_Teams_Attendance_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Upload Attendance File Downloaded from Microsoft Teams
!pip install xlsxwriter

from google.colab import files
from openpyxl import load_workbook
import xlrd 
import xlsxwriter
import pandas as pd
 
 
def check_late(a,b,c): # to check late joinin by relaxation of c minutes
    #a-joined time
    #b-lect start time
    #c-relaxation in mins.
    a=list(a)
    b=list(b)
    
    a[1]=a[1]-c          #subtract c minutes from join time
    a[0]=a[0]+(a[1]//60)
    a[1]=a[1]%60

    if (a[0]<b[0]):
        return 0
    elif (a[0]==b[0]):
        if (a[1]<b[1]):
            return 0
        elif (a[1]==b[1]):
            if (a[2]<=b[2]):
                return 0
            else: 
                return 1
        else:
            return 1
    else:
        return 1                    


def analysis(s_data,l_start,l_end,late_limit): 
    J = str('Joined')
    L = str('Left')
    Jb = str('Joined before')
    l=len(s_data)
    dis=0
    dur=0
    late=0

    t_j=time(s_data[0])
    t_j1=time_join(t_j , l_start)
    t_l=l_end

    #check late join
    if (check_late(t_j1, l_start, late_limit)==0):
        late = 'No Late'
    else:
        late = 'Late'    
    
    dur=0
    ana_data=[s_data[0][0],t_j1,t_l,late,dis,dur]

    if (l==1):
        if (s_data[0][1]==J or s_data[0][1]==Jb):
            t_j=time(s_data[0])
            t_j=time_join(t_j , l_start)
            t_l=l_end
            pr_time=p_time(t_j , t_l)
            dur=pr_time
        else:
            print('data missing')    


    for i in range(1,l):
        
        if (s_data[i-1][0]==s_data[i][0]):    #check name
       
            if ((s_data[i-1][1]==J or s_data[i-1][1]==Jb) and s_data[i][1]==L):    #check join and left 
                             
                t_j=time(s_data[i-1])                
                t_j=time_join(t_j , l_start)               
                t_l=time(s_data[i])                
                t_l=time_left(t_l , l_end)                
                pr_time=p_time(t_j , t_l)    #present time               
                dur = dur + pr_time
                dis=dis+1

                  
    if ((s_data[-1][1]==J or s_data[-1][1]==Jb) and l != 1):    #check join and left 
       
        t_j=time(s_data[-1])
        t_j=time_join(t_j , l_start)
        t_l=l_end
        pr_time=p_time(t_j , t_l)       
        dur = dur + pr_time     
        #dis=dis+1 
    
    if (t_j1[0] != 12):
        jtime=str(t_j1[0]%12) + str(':') + str(t_j1[1]) + str(':') +str (t_j1[2])  
    else :
        jtime=str(t_j1[0]) + str(':') + str(t_j1[1]) + str(':') +str (t_j1[2])
    
    if (t_l[0] != 12):
        ltime=str(t_l[0]%12) + str(':') + str(t_l[1]) + str(':') +str (t_l[2])      
    else: 
        ltime=str(t_l[0]) + str(':') + str(t_l[1]) + str(':') +str (t_l[2])      
 
    dur_min=dur//60
    dur_sec=dur%60
    dur=str(dur_min)+':'+str(dur_sec)
    ana_data=[s_data[0][0],jtime,ltime,late,dis,dur]
    return ana_data

#convert time into tuple(hour,minutes,seconds)
def time(std):

    d_t = std[2]   
    
    t=d_t[d_t.index(',')+2:-2]    
    th=t[:t.index(':')]
    th=int(th)
    mode=str(d_t[-2:])
    if (mode == 'PM' and th != 12):
        th=th+12     
    tm1=t[t.index(':')+1:]    
    tm=tm1[:tm1.index(':')]    
    ts=tm1[tm1.index(':')+1:]    
    
    return((int(th),int(tm),int(ts)))

#access time from csv file
def time1(std):
    #print(std)
    t = std   
    try:
        mode=str(t[-2:])
        #print(mode)
        if (mode == 'am' or mode=='AM' or mode=='pm' or mode=='pm'):
            pass
        else:
            print('please enter am or pm')    
    except:
        print('please enter am or pm')    
    
    try:
        t = t[:-2]
        #print(t)
    except:
        t=t  
        #print(t)  
    
    try :    
        th=t[0:t.index('.')]
    except :
        try:
            th=t[0:t.index(':')]
        except:              
            th=t   
    
    th = int(th)    
    if (mode == 'PM' or mode == 'pm'):
        if (th!=12):
            th=th+12 
    #print(th)
    try:             
        tm=t[t.index('.')+1:]    
       
    except:
        try:
            tm=t[t.index(':')+1:]    
           
        except:            
            tm=0   
    
    return((int(th),int(tm),0))

#present time
def p_time(T1,T2):
    # T1-join time
    # T2-left time
    hr=(T2[0]-T1[0])%12
    min=T2[1]-T1[1]
    sec=T2[2]-T1[2]  
    t = hr*60*60 + min*60 + sec
    return t 
   
#if joined before lecture then assign join time as lecture start time
def time_join(a , b):
    #a - joined time
    #b - lecture start time
    if (a[0]<b[0]):
        return b    
    elif (a[0]>b[0]):
        return a        
    else:
        if (a[1]<b[1]):
            return b
        elif (a[1]>b[1]):
            return a
        else:
            if (a[2]<b[2]):
                return b
            elif (a[2]>b[2]):
                return a
            else:
                return a 

#if left time is after lecture end time then assign left time as lecture end time
def time_left(a , b): 
    #a - left time
    #b - lect end time
    if (a[0] < b[0]):
        return a
    elif (a[0] > b[0]):
        return b
    else:
        if (a[1]<b[1]):
            return a
        elif (a[1]>b[1]):
            return b
        else:
            if (a[2]<b[2]):
                return a
            elif (a[2]>b[2]):
                return b
            else:
                return b     

#Upload excel csv file
check=1
while (check==1):
    try :
        print('-----------------------------------------------')
        uploaded = files.upload()
        for key in uploaded:
            #print(key)
            print('')
        df = pd.read_csv(key,'rb',delimiter="\t",encoding = 'utf-16')

        #df1=df.sort_values(by=['Full Name'])
        df2=df.sort_values(by=['Full Name','Timestamp'])

        a_data1=[]
        a_data=[]
        for index, row in df2.iterrows():    
            a_data.append([row[0],row[1],row[2]])
            a_data1.append([row[0],row[1],row[2]])

        J = str('Joined')
        L = str('Left')
        Jb = str('Joined before')

        for i in range (1,len(df2)): #df2 - sorted data in pandas
            if (a_data1[i-1][0]==a_data1[i][0]):
                if (a_data1[i-1][1]!=L and a_data1[i][1]!=L): #remove consecutive join of same person
                    a_data.remove(a_data1[i])
        check=0            
    except:
        print('Please upload an attendance file downloaded from Microsoft Teams !')
        check=1

In [ ]:
#@title Enter Time

check=1
while (check==1):
    try:
        start = input('Start time (eg 11.15am):')
        end = input('End time (eg 1pm):')
        lect_start = time1(start)
        lect_end = time1(end)
        check=0
    except:
        #print('Enter time in the form (eg 12.15pm):')    
        check=1

check=1
while (check==1):
    try:
        print('Relaxation for Late mark')
        late_limit = int (input('Enter time(in minutes):'))
        check=0
    except:
        print('Please enter integer only !')
        check=1    



count=0
a2_data=a_data.copy()
s_data=[]
final_data=[]

k=0
while (len(a2_data) != 0):
  
    s_data.append(a2_data[0]) 
    a2_data.remove(a2_data[0])
    if (len(a2_data) != 0):
        try:
            while (s_data[0][0] == a2_data[0][0]):   
                s_data.append(a2_data[0])         
                a2_data.remove(a2_data[0])
              
        except:
            pass 

    for itom in s_data:  
     
        if (itom[1]==L): 
            t_l = time(itom)            
            if (t_l[0] < lect_start[0]):
                s_data.remove(itom)
            elif (t_l[0] == lect_start[0]):    
                if (t_l[1] < lect_start[1]):
                    s_data.remove(itom)
                elif (t_l[1] == lect_start[1]):    
                    if (t_l[2] <= lect_start[2]):  
                        s_data.remove(itom)    
   
    final_data.append(analysis(s_data,lect_start,lect_end,late_limit))   
    s_data=[] 

#for i in final_data:
#    print(i)    


workbook = xlsxwriter.Workbook(str(key[:-4])+'Analysis.xlsx')
worksheet0 = workbook.add_worksheet('Analysis')
worksheet1 = workbook.add_worksheet('Original data')

row0 = 0
col0 = 0
worksheet0.write(0, 0 , "Name")
worksheet0.write(0, 1, 'Joined')
worksheet0.write(0, 2 , 'Left')
worksheet0.write(0, 3 , 'Late Mark')
worksheet0.write(0, 4 , 'Discontinued')
worksheet0.write(0, 5 , 'Total Duration')
row0 +=1
for i in range(len(final_data)):
    worksheet0.write(row0, col0, final_data[i][0])
    worksheet0.write(row0, col0+1, final_data[i][1])
    worksheet0.write(row0, col0+2, final_data[i][2])
    worksheet0.write(row0, col0+3, final_data[i][3])
    worksheet0.write(row0, col0+4, final_data[i][4])
    worksheet0.write(row0, col0+5, final_data[i][5])
    row0 +=1

row1 = 0
col1 = 0
worksheet1.write(0, 0 , 'Full Name')
worksheet1.write(0, 1, 'User Action')
worksheet1.write(0, 2 , 'Timestamp')
row1 +=1
for i in range(len(a_data1)):
    worksheet1.write(row1, col1 , a_data1[i][0])
    worksheet1.write(row1, col1+1, a_data1[i][1])
    worksheet1.write(row1, col1+2 , a_data1[i][2])
    row1 +=1
    

workbook.close()
files.download(str(key[:-4])+'Analysis.xlsx')    